In [1]:
#@title 1 - Importação das bibliotecas. Favor apertar [ ▶ ] abaixo

print("######################################################")
print("Instalando e carregando as bibliotecas necessárias...")
print("######################################################\n\n")
# Importando as bibliotecas utilizadas
import pandas as pd
import numpy as np
import progressbar as pb #Biblioteca para visualização da barra de progresso para processos longos 
import warnings
import pickle as pkl
from time import time
import os
from google.colab import files
from google.colab import drive

import warnings

# Biblioteca de ML
## Métricas
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score

## Lib auxiliares
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

## Models
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# Removendo os avisos
warnings.filterwarnings('ignore')

try:
  # Montando o diretório do Drive para leitura de arquivos do programa
  drive.mount('/content/drive')
except:
  print("É necessário permitir que este notebook acesse seus arquivos do Google Drive.")
  print("Favor executar novamente e autorizar o acesso.")

else:
  print("Bibliotecas carregadas! Etapa 1 executada com sucesso!")

def carregaDados(dataset='dados.xlsx'):
  """Função para carregar os dados"""
  df = pd.read_excel(dataset, names=['cod_produto', 'etapa','year', 'month', 'day','hour','minute','valor', 'id_massa'], dtype={'valor':'float'})
  
  return df

def dataPreparation(df):
  """Função para preparação dos dados para uso nas demais etapas"""
  #Criando nova coluna datetime
  df['datetime'] = pd.to_datetime(df[['year','month','day','hour','minute']], yearfirst=True)

  # Dic. atribuindo valores referente a etapa do processo.
  # Critério: números formados por 'XY', onde X referência à "Etapa do processo" e Y à ordem dele
  #      X = 1 -> Etapa de Preparação de massa
  #      X = 2 -> Dados Moldadora
  #      X = 3 -> Dados Moldadora - pós passagem nos cilindro
  #      X = 4 e 5 -> Dados do forno - cozimento
  #      X = 6 -> Dados do forno - pós cozimento

  ordem_etapas = {'A': 1, 'B': 1, 'C': 1, 'D': 1, 'E': 1, 'F': 1, 'G': 1, 'H': 1, 'I': 1, 'J': 1, 'K': 1, 'L': 1, 
              'S.A Farinha de trigo comum doce': 2, 'Quantidade de Fermento': 11, 'Tempo de Batimento 1ª Fase': 12, 
              'Tempo de Batimento 2ª Fase': 13, 'Temperatura de massa': 14, 'VELOCIDADE FORÇADOR': 21, 
              'VELOCIDADE DA MOLDADORA': 22, 'VELOCIDADE LONA MOLDADORA': 23, 'PRESSÃO ROLO ESTRIADO': 24, 
              'PRESSÃO ROLO DE BORRACHA ESQUERDO': 25, 'PRESSÃO ROLO DE BORRACHA DIREITO': 26, 'ALTURA DA FACA': 27, 'VELOCIDADE LONA DE BANDEJA OSCILANTE': 28, 
              'Peso cru': 31, 'Peso cru 1': 31, '[TEMPERATURA ZONA 1]': 41, 'Pressão do teto - zona 1': 42, 
              'Pressão do lastro - zona 1': 43, '[TEMPERATURA ZONA 2]': 44, 'Pressão do teto - zona 2': 45, 
              'Pressão do lastro - zona 2': 46, '[TEMPERATURA ZONA 3]': 47, 'Pressão do teto - zona 3': 48, 'Pressão do lastro - zona 3': 49, 
              '[TEMPERATURA ZONA 4]': 50, 'Pressão do teto - zona 4': 51, 'Pressão do lastro - zona 4': 52, 'Tempo de cozimento': 53, 
              'Largura': 61, 'Largura 1': 61, 'Comprimento': 62, 'Comprimento 1': 62, 'pH': 63, 'Peso assado': 64, 'Peso assado 1': 64, 
              'Cor Hunter L - Teto': 65, 'Umidade (água)': 66, 'Expansão': 67, 'Expansão 1': 67}

  # Criando nova coluna baseado no dicionário ordem etapas criado.
  df['ordem'] = df['etapa'].apply(lambda x : ordem_etapas[x])

  # Ordenando o dataframe pelo valor data/hora
  df.sort_values(['datetime', 'ordem'], inplace=True)

  # Resetando o index
  df.reset_index(drop=True, inplace=True)

  #Removendo o espaço no nome das etapas
  df['etapa'] = df['etapa'].str.lower()
  df['etapa'] = df['etapa'].str.replace(" ", "_")
  df['etapa'] = df['etapa'].str.replace("[", "")
  df['etapa'] = df['etapa'].str.replace("]", "")

  return df

def splitData(df):  
  """Função para adicionar colunas necessárias para enriquecimento dos dados"""
  # Separando DataFrame Processos e Laudo
  ## Dataframe com os dados referente aos Laudos da massa
  df_laudo = df[df['ordem'] == 1].copy()

  ## Dataframe com os dados referente aos Processos
  df_processo = df.loc[df['ordem'] != 1].copy()
  df_processo.sort_values(['datetime'], inplace=True)

  # Preenchendo a coluna id_massa com o valor de S.A Farinha de trigo comum doce
  ## Os valores referente a 'S.A Farinha de trigo comum doce', indicam que aquela massa foi inserida na linha de produção àquela hora. 
  ## Dessa forma, podemos preencher a coluna id_massa com os mesmos valores de 'S.A Farinha de trigo comum doce', para todas as etapas do horário que ele foi inserido, até o próximo registro válido.
  df_processo['id_massa'] = df_processo['valor'][df_processo['ordem'] == 2]

  #Preenchendo os valores nulos com o último valor válido
  ## Como os valores estão ordenado por processo e data/hora, e o id_massa é o primeiro a ser registrado no início do processo, logo podemos preencher todos os valores seguintes com o mesmo id_massa até que um próximo id_massa seja inserido na linha.
  df_processo['id_massa'].fillna(method='ffill', inplace=True)

  return df_processo, df_laudo

def dataCleaningProcess(df_processo):
  """Função para limpar/transformar dados do dataset de Processos"""
  #Removendo as linhas etapa 'S.A Farinha de trigo comum doce'
  df_processo = df_processo.loc[df_processo['ordem'] != 2]
  # Removendo colunas desnecessárias 
  df_processo.drop(labels=['year','month','day','hour','minute'], axis=1, inplace=True)
  #Removendo os valores nulos
  df_processo.dropna(inplace=True)
  # Transformando a coluna id_massa para o tipo int
  df_processo['id_massa'] = df_processo['id_massa'].astype(int)

  return df_processo

def dataCleaningLaudo(df_laudo):
  """Função para limpar/transformar dados do dataset de Laudo"""
  # Removendo "SA ' dos valores de S.A
  df_laudo['id_massa_temp'] = df_laudo['id_massa'].str.strip('SAsa ')

  # Recuperando os valores perdidos no lstrip
  df_laudo['id_massa_temp'].fillna(df_laudo['id_massa'], inplace=True)

  # Copiando os valores para a coluna id_massa
  df_laudo['id_massa'] = df_laudo['id_massa_temp'].astype('int').copy()

  # Removendo colunas desnecessárias 
  df_laudo.drop(labels=['year','month','day','hour','minute', 'id_massa_temp', 'datetime'], axis=1, inplace=True)

  # Agrupando os valores de Laudo por id_massa
  df_laudo = df_laudo.groupby(['id_massa', 'etapa']).mean().reset_index().copy()

  # Pivotando os valores agrupados
  df_laudo = pd.pivot(df_laudo, index=['id_massa', 'cod_produto'], columns=['etapa'], values='valor').reset_index()

  # #Preenchendo valores para a coluna L conforme fórmula
  df_laudo['l'] = round((df_laudo['k'] / df_laudo['a']), 3)

  # # Lista das colunas de interesse
  prop_laudo = ['a', 'b', 'c', 'd', 'e','f', 'g', 'h', 'i', 'j', 'k', 'l']

  # Retornando o df_laudo originário com as propriedades na coluna etapa
  df_laudo = df_laudo.melt(id_vars=['id_massa'], value_vars=prop_laudo, value_name='valor')

  # Renomeando a coluna conforme padrão
  df_laudo.rename(columns={"variable": "etapa"}, inplace=True)

  return df_laudo

def prepareDataFeature(df_processo, df_laudo):
  """Função que prepara os dados de processo (features) para a separação de amostras"""
  # Remover os dados de df_processo que possuem id_massa inexistente em df_laudo
  id_massa_unicos = list(df_laudo['id_massa'])
  df_feature = df_processo[df_processo['id_massa'].isin(id_massa_unicos)]

  # Separando as colunas de interesse
  df_feature = df_feature[['id_massa', 'etapa', 'valor', 'ordem', 'datetime']].loc[df_feature['ordem'] != 67]

  # agrupar dados de lados diferentes da esteira numa mesma iteração pela média
  df_feature = df_feature.groupby(['ordem','datetime']).mean().reset_index().sort_values(['datetime','ordem'])

  return df_feature

def prepareDataTarget(df_processo):
  """Função que prepara os dados de expansão (variável target) para a separação de amostras"""
  # Seprando as colunas de interesse
  df_target = df_processo[['id_massa', 'etapa', 'valor', 'ordem', 'datetime']].loc[df_processo['ordem'] == 67]

  # Agrupamento dos valores de expansão numa mesma iteração pela média
  df_target = df_target.groupby(['datetime']).mean().reset_index()
  
  return df_target

def prepareEtapa():
  """Função que prepara um dataframe com as etapas para a separação de amostras"""

  ordem_etapas = {'quantidade_de_fermento': 11, 'tempo_de_batimento_1ª_fase': 12, 'tempo_de_batimento_2ª_fase': 13, 'temperatura_de_massa': 14, 
              'velocidade_forçador': 21, 'velocidade_da_moldadora': 22, 'velocidade_lona_moldadora': 23, 'pressão_rolo_estriado': 24, 
              'pressão_rolo_de_borracha_esquerdo': 25, 'pressão_rolo_de_borracha_direito': 26, 'altura_da_faca': 27, 'velocidade_lona_de_bandeja_oscilante': 28, 
              'peso_cru': 31, 'temperatura_zona_1': 41, 'pressão_do_teto_-_zona_1': 42, 'pressão_do_lastro_-_zona_1': 43, 'temperatura_zona_2': 44, 
              'pressão_do_teto_-_zona_2': 45, 'pressão_do_lastro_-_zona_2': 46, 'temperatura_zona_3': 47, 'pressão_do_teto_-_zona_3': 48, 'pressão_do_lastro_-_zona_3': 49, 
              'temperatura_zona_4': 50, 'pressão_do_teto_-_zona_4': 51, 'pressão_do_lastro_-_zona_4': 52, 'tempo_de_cozimento': 53, 
              'largura': 61, 'comprimento': 62, 'ph': 63, 'peso_assado': 64, 'cor_hunter_l_-_teto': 65, 'umidade_(água)': 66, 'expansão': 67}

  # Criação do df_etapa para o algoritmo da função criada
  df_etapa = pd.DataFrame.from_dict(ordem_etapas, orient='index').sort_values(by=0)
  df_etapa.rename(columns={0:'ordem'},inplace=True)
  df_etapa = df_etapa.reset_index()
  df_etapa.rename(columns={'index':'etapa'},inplace=True)

  #Removendo valores de ordem 1 e 2
  cond1 = df_etapa['ordem'] != 1
  cond2 = df_etapa['ordem'] != 2

  df_etapa = df_etapa.loc[cond1 & cond2].reset_index(drop=True)

  return df_etapa

def prepareLaudo(df_laudo, df_target):
  """Função que prepara os dados de Laudo para a separação de amostras"""
  # Remover os dados de df_processo que possuem id_massa inexistente em df_laudo
  id_massa_unicos = list(df_target['id_massa'].unique())
  df_laudo = df_laudo[df_laudo['id_massa'].isin(id_massa_unicos)]

  # Separando as colunas de interesse
  df_laudo = df_laudo[['id_massa', 'etapa', 'valor']]

  return df_laudo

def get_ids(df_feature, df_etapa, df_target, df_laudo):
  """Função que organiza valor de cada etapa do processo e laudo, para uma dada expansão"""
   # dataframe que será preenchido e retornado
  df_sorted = pd.DataFrame({'id_iteracao': [], 'id_massa': [], 'datetime': [], 'etapa': [], 'ordem': [], 'valor': []})
  id = 1
   
   # dataframe com os valores elegiveis dentro da tolerância de cada iteração para adicionar valores e pegar o mais próximo do tempo da variavel target
  elegiveis = pd.DataFrame({'ordem': [], 'datetime': [], 'id_massa': [], 'valor': [], 'timediff': []})
  # para cada linha de df_target preencher as etapas disponíveis no df_feature com base no df_etapa:
  for id_target, row_target in df_target.iterrows():
      id_massa_atual = row_target['id_massa']
      for id_etapa, row_etapa in df_etapa.iterrows():
          # se for a etapa da variável target preencher com dados da variável target
          if row_etapa['ordem'] == 67:
              df_sorted = df_sorted.append({'id_iteracao': id, 'id_massa': row_target['id_massa'], 'datetime': row_target['datetime'],
               'etapa': row_etapa['etapa'], 'ordem': row_etapa['ordem'], 'valor': row_target['valor']}, ignore_index=True)
          
          else:
              for id_feat, row_feat in df_feature.iterrows():
                                     
                  if row_feat['id_massa'] == row_target['id_massa']:
                      # se datetime de feature for dentro do prazo de tolerancia e se a ordem do df_etapa for a mesma que df_feature, adicionar aos elegíveis
                      timediff = (row_target['datetime']-row_feat['datetime']).total_seconds()/60
                      
                      if (timediff in range(0, 31) and row_etapa['ordem'] == row_feat['ordem'] and row_feat['ordem'] >= 40):
                          elegiveis = elegiveis.append({'ordem': row_feat['ordem'], 'datetime': row_feat['datetime'],
                                                      'id_massa': row_feat['id_massa'], 'valor': row_feat['valor'], 'timediff': timediff}, ignore_index=True)
                          df_feature.drop(index=id_feat, axis=0, inplace=True)
                      elif (timediff in range(25, 51) and row_etapa['ordem'] == row_feat['ordem'] and row_feat['ordem'] in range(20, 40)):
                          elegiveis = elegiveis.append({'ordem': row_feat['ordem'], 'datetime': row_feat['datetime'],
                                                      'id_massa': row_feat['id_massa'], 'valor': row_feat['valor'], 'timediff': timediff}, ignore_index=True)
                          df_feature.drop(index=id_feat, axis=0, inplace=True)
                      elif (timediff in range(35, 65) and row_etapa['ordem'] == row_feat['ordem'] and row_feat['ordem'] < 20):
                          elegiveis = elegiveis.append({'ordem': row_feat['ordem'], 'datetime': row_feat['datetime'],
                                                      'id_massa': row_feat['id_massa'], 'valor': row_feat['valor'], 'timediff': timediff}, ignore_index=True)
                          df_feature.drop(index=id_feat, axis=0, inplace=True)
              
              # se elegiveis for vazio, preecher df_sorted com "-"
              if len(elegiveis) == 0:
                  df_sorted = df_sorted.append({'id_iteracao': id, 'id_massa': np.nan, 'datetime': np.nan,
                                               'etapa': row_etapa['etapa'], 'ordem': row_etapa['ordem'], 'valor': np.nan}, ignore_index=True)
              
              # caso contrário preencher com o valor de df_feature com menor timediff
              else:
                  # reordenar elegiveis para pegar o menor timediff
                  elegiveis = elegiveis.sort_values(['timediff']).reset_index()
                  df_sorted = df_sorted.append({'id_iteracao': id, 'id_massa': elegiveis['id_massa'][0], 'datetime': elegiveis['datetime'][0],
                   'etapa': row_etapa['etapa'], 'ordem': row_etapa['ordem'], 'valor': elegiveis['valor'][0]}, ignore_index=True)
          # fechado uma iteração, limpar o elegiveis
          elegiveis = pd.DataFrame({'ordem': [], 'datetime': [], 'id_massa': [], 'valor': [], 'timediff': []})
      
      # incluir dados da massa na iteração
      for id_massa, row_massa in df_laudo[df_laudo['id_massa'] == row_target['id_massa']].iterrows():
          df_sorted = df_sorted.append({'id_iteracao': id, 'id_massa': row_target['id_massa'], 'datetime': row_target['datetime'],
           'etapa': row_massa['etapa'], 'ordem': row_massa['etapa'], 'valor': row_massa['valor']}, ignore_index=True)
      #Remover todos os valores de df_feature com id_massa diferentes
              
      if row_target['id_massa'] == id_massa_atual:
          # incrementar o id para a próxima iteração
          id += 1
      else:
          df_feature = df_feature.loc[df_feature['id_massa'] != id_massa_atual]
          id += 1
  
  return df_sorted

######### FUNÇÕES PARA CRIAÇÃO DO MODELO

def fill_values(colunas, df):
  '''Função para preencher todos os valores nulos nas colunas de feature do processo, onde: 
  os primeiro são iguais ao primeiro valor existente, 
  e os demais valores são preenchidos com a média entre os valores: [anterior + seguinte] / 2'''
  
  for col in colunas:
    index_Not_Null =list(df[col].loc[df[col].isna() == False].index)

    inicial =  index_Not_Null[0]
    
    # Preenchendo todos os valores nulos com o primeiro valor preenchido
    if index_Not_Null[0] != 0:
      df[col][:inicial] = df[col].iloc[inicial]

    # Preenchendo todos os valores nulos com a média entre o valor anterior e o próximo valor preenchido
    tam = len(index_Not_Null)
    A = 0
    B = 0
    
    for i in range(0, tam-1):
        passo = abs(index_Not_Null[i] - index_Not_Null[i+1])

        if passo > 1:
          
            A = index_Not_Null[i]
            B = index_Not_Null[i+1]
            df[col][A+1:B] = (df[col][A] + df[col][B]) / 2
    
    # Preenchendo valores finais com o último valor preenchido
    df.fillna(method='ffill', inplace=True)

def compare_models(x_train_scaled, x_test_scaled, y_train, y_test, model_list = [LinearRegression()]):
  """Função que retorna o melhor modelo obtido e o modelo de Regressão Linear"""
  r2_rank = []
  model_rank = []
  model_dict = {}

  for model in model_list:
      start = time()
      model.fit(x_train_scaled, y_train)
      train_time = time() - start
      start = time()
      y_pred = model.predict(x_test_scaled)
      predict_time = time()-start
      r2 = r2_score(y_test, y_pred)
      model_rank.append(model)
      r2_rank.append(r2)
      model_dict[model] = (train_time, predict_time,
                          explained_variance_score(y_test, y_pred), 
                          mean_absolute_error(y_test, y_pred),
                          r2_score(y_test, y_pred))

  # Encontrando o modelo com maior R2
  max_r2 = max(r2_rank)
  max_index = r2_rank.index(max_r2)

  # Registro dos parâmetros estatísticos obtidos no melhor modelo
  model_stats = {}
  model_stats[model_rank[max_index]] = model_dict[model_rank[max_index]]

  return model_rank[max_index], model_stats

def make_data_model_format(df):

    # Removendo o valor .0 na coluna ordem para facilitar a remoção
    df['ordem'] = df['ordem'].str.replace('.0', '')

    # Removendo os valores de ordem = 1 e 2
    # 1 = Dados de Laudo incompletos
    # 2 = Dados de S.A Farinha
    filtro1 = df['ordem'] != '1'
    filtro2 = df['ordem'] != '2'
    df = (df.loc[filtro1 & filtro2])

    # Preenchendo id_massa nulos
    df['id_massa'].fillna(method="bfill", inplace=True)

    # Alterando o tipo dos dados da coluna valor
    df['valor'] = df['valor'].astype('float32')

    # Gerando novo df com os dados de etapa pivotados
    reformated_data = pd.pivot_table(df,values='valor',index=['id_iteracao', 'id_massa'], columns=['etapa']).reset_index()

    colunas = list(reformated_data.drop(['id_iteracao','id_massa'], axis=1).columns)

    fill_values(colunas, reformated_data)

    return reformated_data

def find_better_model(df_model, X, y):
  '''Função para encontrar modelo com melhor R2 score'''

  # Separação dos dados de treino e teste
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Aplicando StandardScaler nos dados de treino e teste
  sc = StandardScaler()
  sc.fit(X_train)
  
  # Transformando os dados conforme StandardScaler obtido
  x_train_scaled = sc.transform(X_train)
  x_test_scaled = sc.transform(X_test)

  # Definindo dos modelos que serão testados
  regressors = [
  LinearRegression(),
  Ridge(),
  RandomForestRegressor(),
  DecisionTreeRegressor()
  ]

  # Obtendo o melhor modelo testado
  best_testing_models, model_stats = compare_models(x_train_scaled, x_test_scaled, y_train, y_test, regressors)
  
  # Prints com as estatísticas do modelo com maior R2
  print("############################")
  print(f"O modelo que obteve melhor performance foi: {best_testing_models}\n")
  print(f"\t- Training time: {round(model_stats[best_testing_models][0], 3)}s")
  print(f"\t- Prediction time: {round(model_stats[best_testing_models][1],3)}s")
  print(f"\t- Explained variance: {round(model_stats[best_testing_models][2], 3)}")
  print(f"\t- Mean absolute error: {round(model_stats[best_testing_models][3],3)}")
  print(f"\t- R2 score: {round(model_stats[best_testing_models][4], 3)}")
  print("############################\n")
  
  return best_testing_models, model_stats

def make_model(df_model):
  '''Função para criar modelo de previsão a partir do melhor R2 score obtido'''

  # Eliminando as colunas não utilisadas no modelo
  modeling_data = df_model.drop(columns=['id_iteracao', 'id_massa'])

  # Separação de features e variável target
  colunas = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 
            'quantidade_de_fermento', 'tempo_de_batimento_1ª_fase', 'tempo_de_batimento_2ª_fase', 'temperatura_de_massa',
            'peso_cru', 'altura_da_faca','pressão_rolo_de_borracha_direito', 'pressão_rolo_de_borracha_esquerdo','velocidade_da_moldadora',
            'velocidade_forçador', 'velocidade_lona_moldadora', 'pressão_rolo_estriado',
            'tempo_de_cozimento', 'pressão_do_lastro_-_zona_1', 'pressão_do_lastro_-_zona_2', 'pressão_do_lastro_-_zona_3', 'pressão_do_lastro_-_zona_4',
            'pressão_do_teto_-_zona_1', 'pressão_do_teto_-_zona_2', 'pressão_do_teto_-_zona_3', 'pressão_do_teto_-_zona_4',
            'temperatura_zona_1', 'temperatura_zona_2', 'temperatura_zona_3', 'temperatura_zona_4',
            'comprimento','largura', 'peso_assado', 'umidade_(água)', 'ph', 'cor_hunter_l_-_teto']

  X = modeling_data.loc[:,colunas] #Definindo colunas de interesse e ordenação necessária
  y = modeling_data[['expansão']]
  
  best_testing_models, model_stats = find_better_model(df_model, X, y)

  # Aplicando StandardScaler nos dados de treino e teste
  sc_fit = StandardScaler()
  sc_fit.fit(X)
  
  # Transformando os dados conforme StandardScaler obtido
  X_scaled = sc_fit.transform(X)

  # Treinando o melhor modelo obtido com toda a base de dados
  the_best_model = best_testing_models.fit(X_scaled, y)

  return the_best_model, sc_fit, model_stats

def make_lr_model(df_model):
  '''Função para criar modelo de Regressão Linear para otimização dos parâmetros'''

  # Eliminando as colunas não utilisadas no modelo
  modeling_data = df_model.drop(columns=['id_iteracao', 'id_massa'])

  # Separação de features e variável target
  colunas = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 
            'quantidade_de_fermento', 'tempo_de_batimento_1ª_fase', 'tempo_de_batimento_2ª_fase', 'temperatura_de_massa',
            'peso_cru', 'altura_da_faca','pressão_rolo_de_borracha_direito', 'pressão_rolo_de_borracha_esquerdo','velocidade_da_moldadora',
            'velocidade_forçador', 'velocidade_lona_moldadora', 'pressão_rolo_estriado',
            'tempo_de_cozimento', 'pressão_do_lastro_-_zona_1', 'pressão_do_lastro_-_zona_2', 'pressão_do_lastro_-_zona_3', 'pressão_do_lastro_-_zona_4',
            'pressão_do_teto_-_zona_1', 'pressão_do_teto_-_zona_2', 'pressão_do_teto_-_zona_3', 'pressão_do_teto_-_zona_4',
            'temperatura_zona_1', 'temperatura_zona_2', 'temperatura_zona_3', 'temperatura_zona_4',
            'comprimento', 'largura', "expansão", 'umidade_(água)']

  X = modeling_data.loc[:,colunas] #Definindo colunas de interesse e ordenação necessária
  y = modeling_data[['peso_assado']]

  # Obtendo StandardScaler nos dados em X
  sc_linear_fit = StandardScaler()
  sc_linear_fit.fit(X)
  
  # Transformando os dados conforme StandardScaler obtido
  X_scaled = sc_linear_fit.transform(X)

  # Treinando um modelo de Regressão Linear com toda a base de dados
  lr = LinearRegression()
  linear_regression_model = lr.fit(X_scaled, y)
 
  return linear_regression_model, sc_linear_fit

def create_model_pickle(prediction_model=[], linear_model=[], model_stats=dict()):
    # Cria um objeto iterador com permissão de escrita - model.pkl
    ## Primeiro objeto é o melhor modelo obtido e StandardScale obtido
    ## Segundo objeto é o modelo de Linear Regression e StandardScale obtido
    ## Terceiro objeto são as estatísticas de performance do melhor modelo obtido
    with open('/content/drive/MyDrive/DNC_PredicaoExpansao/pkl_models/model_pkl', 'wb') as model_files:
        pkl.dump((prediction_model[0], linear_model[0], model_stats), model_files)
    print("Modelo para predição criado com sucesso !")

    with open('/content/drive/MyDrive/DNC_PredicaoExpansao/pkl_models/sc_pkl', 'wb') as sc_files:
        pkl.dump((prediction_model[1], linear_model[1]), sc_files)
    print("Standard Scale criado com sucesso !")


def load_data_model(data='df_iteracoes.csv'):
    ## Criar condicional que verifica se o arquivo df_iteracoes.csv existe na pasta corrente

    if data == "/content/drive/MyDrive/DNC_PredicaoExpansao/Input/df_iteracoes.csv" :
      # Apontando para a pasta do dataset, conforme padrão do projeto
      dataset_model = "/content/drive/MyDrive/DNC_PredicaoExpansao/Input/df_iteracoes.csv"

      # Carregando os dados
      df = pd.read_csv(data, index_col='Unnamed: 0')
      return df
    else:
      print("Verifique se o arquivo: df_iteracoes.csv está salvo corretamente na pasta ../Input")

######################################################
Instalando e carregando as bibliotecas necessárias...
######################################################


Mounted at /content/drive
Bibliotecas carregadas! Etapa 1 executada com sucesso!


---

In [2]:
#@title 2 - Carregar funções para processar novo modelo preditor. Pressione [ ▶ ] abaixo

try:
  file_name = "dados.xlsx"

  if file_name not in os.listdir():
    uploaded = files.upload()
    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(uploaded[fn])))  
except NameError:
  print("Verifique se a etapa 1 foi executada corretamente.")

else:
  try:
    df_processo, df_laudo = splitData(dataPreparation(carregaDados(file_name)))
    df_processo = dataCleaningProcess(df_processo)
    df_laudo = dataCleaningLaudo(df_laudo)

    df_feature = prepareDataFeature(df_processo, df_laudo)
    df_target = prepareDataTarget(df_processo)
    df_etapa = prepareEtapa()
    df_laudo = prepareLaudo(df_laudo, df_target)

    print("Funções carregadas! Etapa 2 executada com sucesso!")

  except NameError:
    print("Verifique se a etapa 1 foi executada corretamente.")
  except:
    print("Verifique se o arquivo 'dados.xlsx' foi carregado e execute novamente.")

Funções carregadas! Etapa 2 executada com sucesso!


---

In [ ]:
#@title 3 - Gerar novo modelo preditor. Pressione [ ▶ ] abaixo

try:
  # # gerar o df_sorted com a ordenação de cada iteração
  df_sorted = get_ids(df_feature, df_etapa, df_target, df_laudo)

  df_sorted = read_csv("/content/drive/MyDrive/DNC_PredicaoExpansao/Input/df_iteracoes.csv")                                  # Apontar o caminho para /Input/df_iteracoes.csv

  # # Salvando df_sorted em um arquivo externo
  df_sorted.to_csv("/content/drive/MyDrive/DNC_PredicaoExpansao/Input/df_iteracoes.csv")                                      # Apontar o caminho para /Input/df_iteracoes.csv
  df_sorted.drop('Unnamed: 0', axis=1).to_excel("/content/drive/MyDrive/DNC_PredicaoExpansao/Output/dados_por_amostra.xlsx")  # Apontar o caminho para /Output/dados_por_amostra.xlsx

  # Gerando o modelo
  modelfile = "/content/drive/MyDrive/DNC_PredicaoExpansao/Input/df_iteracoes.csv"                                              # Apontar o caminho para /Input/df_iteracoes.csv
  model_data = load_data_model(modelfile)
  df_model = make_data_model_format(model_data)
  best_model, sc, model_stats = make_model(df_model)
  linear_regression_model, sc_lr = make_lr_model(df_model)
  create_model_pickle([best_model, sc], [linear_regression_model, sc_lr], model_stats)

except NameError:
  print("Verifique se as etapas 1 e 2 foram executadas corretamente.")
  print("Se necessário, vá em: \n\t>>'Ambiente de execução' >> 'Reiniciar ambiente de execução' OU pressione Ctrl+M ")

Verifique se as etapas 1 e 2 foram executadas corretamente.
Se necessário, vá em: 
	>>'Ambiente de execução' >> 'Reiniciar ambiente de execução' OU pressione Ctrl+M 
